This is my first public kernel and I would really appreciate your feedback.

The dataset contains 6000 images of banknote each of shape ( 1280, 720, 3 ). Each banknote category is divided into folders. Our task is to build a model which recognize the banknote.

Import data, preprocess the images and manage their label

In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2

import os

from PIL import Image

X = []

labels = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        
        if( len( str( os.path.join( dirname, filename ) ).split( '/' ) ) > 5 ) :
            
            image = cv2.imread( os.path.join( dirname,filename ) )
            
            img_array = Image.fromarray( image, 'RGB' )
            
            resized_img = img_array.resize( ( 64, 64 ) )
            
            X.append( np.array( resized_img ) ) 
            
            label = str( os.path.join( dirname,filename ) ).split( '/' )[-2]
            
            if( label == '5' ) :
                
                labels.append( 0 )
                
            elif( label == '10' ) :
                
                labels.append( 1 )
             
            elif( label == '20' ) :
                
                labels.append( 2 )
                
            elif( label == '50' ) :
                
                labels.append( 3 )
                
            elif( label == '100' ) :
                
                labels.append( 4 )
                
            else :
                
                labels.append( 5 )
            
# Any results you write to the current directory are saved as output.


In [25]:
print( 'Number of images : %d' % len( X ) )

print( 'Associated labels : %d' % len( labels ) )

Number of images : 6000
Associated labels : 6000


In [26]:
X = np.array( X )

print( 'Shape of X : ', X.shape )

Shape of X :  (6000, 64, 64, 3)


In [27]:
Y = np.array( labels )

Y = np.reshape( Y, ( 6000, 1 ) )

print( 'Shape of Y : ', Y.shape )

Shape of Y :  (6000, 1)


split dataset into train and test set

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split( X, Y, random_state = 10, test_size = 0.2 )

In [29]:
print( X_train.shape )

print( Y_train.shape )

(4800, 64, 64, 3)
(4800, 1)


Change Y_train into categorical

In [30]:
from keras.utils import to_categorical

In [32]:
Y_train = np.reshape( Y_train, ( 4800, 1 ) )

Y_train = to_categorical( Y_train, 6 )

Y_train.shape

(4800, 6)

create model

In [33]:
from keras import Sequential

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, ZeroPadding2D, AveragePooling2D

from keras.layers.advanced_activations import LeakyReLU

from keras.layers.normalization import BatchNormalization

In [34]:
from keras import backend as K

K.clear_session()

model = Sequential()

model.add( ZeroPadding2D( input_shape = ( 64, 64, 3 ), padding = ( 3, 3 ) ) )

model.add ( Conv2D( 32, ( 7, 7 ), strides = (  1, 1 ) ) )

model.add( BatchNormalization( axis = 3 ) )
 
model.add( Activation( 'relu' ) )

model.add( MaxPooling2D( ( 2, 2 ) ) )

model.add( Flatten() )

model.add( Dense( 6 ) )

model.add( Activation( 'softmax' ) )

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                

In [35]:
model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )

model training

In [36]:
model.fit( X_train, Y_train, batch_size = 64, epochs = 20 )

Epoch 1/20
4800/4800 [==============================] - 1s 242us/step - loss: 2.0161 - accuracy: 0.6035
Epoch 2/20
4800/4800 [==============================] - 1s 183us/step - loss: 0.4340 - accuracy: 0.8544
Epoch 3/20
4800/4800 [==============================] - 1s 190us/step - loss: 0.1953 - accuracy: 0.9346
Epoch 4/20
4800/4800 [==============================] - 1s 197us/step - loss: 0.0841 - accuracy: 0.9717
Epoch 5/20
4800/4800 [==============================] - 1s 182us/step - loss: 0.0651 - accuracy: 0.9762
Epoch 6/20
4800/4800 [==============================] - 1s 181us/step - loss: 0.1018 - accuracy: 0.9644
Epoch 7/20
4800/4800 [==============================] - 1s 188us/step - loss: 0.1057 - accuracy: 0.9631
Epoch 8/20
4800/4800 [==============================] - 1s 186us/step - loss: 0.0946 - accuracy: 0.9685
Epoch 9/20
4800/4800 [==============================] - 1s 192us/step - loss: 0.0812 - accuracy: 0.9754
Epoch 10/20
4800/4800 [==============================] - 1s 187u

prediction on test set

In [37]:
Y_predict = model.predict_classes( X_test )

accuracy on test set

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score( Y_test, Y_predict )

0.99

In [41]:
from sklearn.metrics import classification_report

print( classification_report( Y_test, Y_predict ) )

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       198
           1       0.98      0.99      0.98       197
           2       1.00      1.00      1.00       213
           3       0.98      0.98      0.98       194
           4       1.00      1.00      1.00       215
           5       1.00      0.98      0.99       183

    accuracy                           0.99      1200
   macro avg       0.99      0.99      0.99      1200
weighted avg       0.99      0.99      0.99      1200

